In [33]:
import caffe as cf
import cPickle as pickle
from utils_birds import loadSamples
import numpy as np
from caffe import layers as cfL
from caffe import params as cfP
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline


In [63]:
len(trainIdx)

120

In [100]:
#reading the extracted feats of alexnet
with(open('/home/jsotaloram/active_learning/exp_birds/feats_alexnet_birdsDS.pkl','rb')) as f:
    birds_data = pickle.load(f)
#birds_data contains alexNet feats fc6 and fc7 for all 600 samples: 
#Let's compute the performance measures, first partitionate as required
samples = loadSamples()
splitSamples = np.loadtxt("splitSamples.txt", str, delimiter=';')
birdsFolder = "/data1/birds"


woodSamples = np.array(splitSamples[1:, 0], dtype=int) -1
egretSamples = 100*1 + np.array(splitSamples[1:, 1], dtype=int) - 1
owlSamples = 100*2 + np.array(splitSamples[1:, 2], dtype=int) - 1
toucanSamples = 100*3 + np.array(splitSamples[1:, 3], dtype=int) - 1
puffinSamples = 100*4 + np.array(splitSamples[1:, 4], dtype=int) - 1
mandarinSamples = 100*5 + np.array(splitSamples[1:, 5], dtype=int) - 1
trainIdx = np.concatenate((
        woodSamples[0:20],
        egretSamples[0:20],
        owlSamples[0:20],
        toucanSamples[0:20],
        puffinSamples[0:20],
        mandarinSamples[0:20],
        ))

crossIdx = np.concatenate((
        woodSamples[20:50],
        egretSamples[20:50],
        owlSamples[20:50],
        toucanSamples[20:50],
        puffinSamples[20:50],
        mandarinSamples[20:50],
        ))

testIdx = np.concatenate((
        woodSamples[50:100],
        egretSamples[50:100],
        owlSamples[50:100],
        toucanSamples[50:100],
        puffinSamples[50:100],
        mandarinSamples[50:100],
        ))


XTrainLfc7 = [birds_data[1][i] for i in trainIdx]
Xtrain_fxix = np.array([birds_data[1][i] for i in range(len(trainIdx))])

yTrainLfc7 = samples[trainIdx, 0]
yTrainLfc7_fxix = [samples[i,0] for i in range(len(trainIdx))]

XCrossLfc7 =  [birds_data[1][i] for i in crossIdx]
yCrossLfc7 = samples[crossIdx, 0]

XTestLfc7 =  [birds_data[1][i] for i in testIdx]
XTestLfc7_fxix = np.array([birds_data[1][i] for i in range(300,600)])

yTestLfc7 = samples[testIdx, 0]
yTestLfc7_fxix =  [birds_data[1][i] for i in range(300,600)]
birdsLabels = os.listdir(birdsFolder)


In [110]:
len(XTrainLfc7)

120

In [103]:
samples

dict_cats_nameOnly = {0:'wood_duck', 1:'egret',2:'owl', 3:'toucan',4:'mandarin',5:'puffin'}

inv_bird_map_nameOnly = {v: k for k, v in dict_cats_nameOnly.items()}

all_samples_cats = []
for sample in samples:
    all_samples_cats.append(inv_bird_map_nameOnly[sample[0]])
len(all_samples_cats)

yTrainLfc6_labels = [all_samples_cats[i] for i in trainIdx]
yCrossLfc6_labels = [all_samples_cats[i] for i in crossIdx]
yTestLfc6_labels =  [all_samples_cats[i] for i in testIdx]
len(yTrainLfc6_labels)


120

In [104]:

len(yTestLfc6)

#AssertionErrorting that we are mapping good the name to the category
yTestLfc6[0] == dict_cats_nameOnly[yTestLfc6_labels[0]]
#yTestLfc6_labels

True

In [105]:
type(Xtrain_fxix)

numpy.ndarray

In [106]:
#batch_size = 20
solverPrototxt='/home/jsotaloram/active_learning/exp_birds/solver_s_birds.prototxt'
deployPrototxt='/home/jsotaloram/active_learning/exp_birds/train_birds_small.prototxt'
cf.set_mode_gpu()
#active_birds_net = cf.Net(deployPrototxt, cf.TRAIN)

In [107]:
#all_shapes = [(k, v.data.shape) for k, v in active_birds_net.blobs.items()]
#num_hidden_neurons = all_shapes[2][1][1] #the first dimension of shape of fc7 layer
num_hidden_neurons = 32

In [108]:
train_size = 120
val_size = 180
test_size = 300
data4D = np.zeros([train_size ,1,1,4096],np.float32)
data4DL = np.zeros([train_size ,1,1,1], np.float32)
data4D_val = np.zeros([val_size ,1,1,4096],np.float32)
data4DL_val = np.zeros([val_size ,1,1,1], np.float32)
data4D_test = np.zeros([test_size ,1,1,4096],np.float32)
data4DL_test = np.zeros([test_size ,1,1,1], np.float32)

full_idx_range = np.arange(train_size)
full_idx_range_val = np.arange(val_size)
full_idx_range_test = np.arange(test_size)

#print(full_idx_range)
np.random.shuffle(full_idx_range)
np.random.shuffle(full_idx_range_val)
np.random.shuffle(full_idx_range_test)

#print(full_idx_range)

Xrs = np.array(XTrainLfc7).reshape(train_size,1,1,4096)
Yrs = np.array(yTrainLfc6_labels).reshape(train_size,1,1,1)

#Xval = XCrossLfc7.reshape(val_size,1,1,4096)
#Yval = np.array(yCrossLfc6_labels).reshape(val_size,1,1,1)

Xtest = np.array(XTestLfc7).reshape(test_size,1,1,4096)
Ytest = np.array(yTestLfc6_labels).reshape(test_size,1,1,1)

data4D[0:120,:,:,:]  = [Xrs[train_idx] for train_idx in full_idx_range]
data4DL[0:120,:,:,:] = [Yrs[train_idx] for train_idx in full_idx_range]

data4D_val[0:180,:,:,:]  = [Xval[val_idx] for val_idx in full_idx_range_val]
data4DL_val[0:180,:,:,:] = [Yval[val_idx] for val_idx in full_idx_range_val]

data4D_test[0:300,:,:,:]  = [Xtest[test_idx] for test_idx in full_idx_range_test]
data4DL_test[0:300,:,:,:] = [Ytest[test_idx] for test_idx in full_idx_range_test]

print Xrs.shape
#print data4DL
#print yTrainLfc6_labels

#active_birds_net.blobs['data'].data[...] = data4D[0:10] #hardcoding the first batch
#active_birds_net.blobs['label'].data[...] = data4DL[0:10]#yTrainLfc6_labels[0:10]


AttributeError: 'list' object has no attribute 'reshape'

In [112]:
yTrainLfc6_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [89]:
lines_solver = []
save_protos_path = '/home/jsotaloram/active_learning/exp_birds/protos/'
lines_proto = []
f = open(deployPrototxt,'r')
for line in f:
    lines_proto.append(line)
    
[line_p for line_p in lines_proto[0:10]]
f = open(solverPrototxt,'r')
for line in f:
    lines_solver.append(line)

learning_rates_pool = [0.0001,0.0003, 0.001,0.005,0.01,0.1,0.5,0.8,1,2,4]
decay_mult = [0.0001,0.001,0.005, 0.01,0.1,0.5, 1,5,2,10,100]
hyper_params_combs = [] 
for lr_item in learning_rates_pool:
    for dec_item in decay_mult:
        #print 'combination of lr: ' + str(lr_item) +' and decay: ' +str(dec_item) + '\n'
        hyper_params_combs.append((lr_item,dec_item))

for (lr_i,wd_i) in hyper_params_combs:
    print 'generating prototxt for LR: ' + str(lr_i) + ' and WD: ' + str(wd_i)
    temp_proto = open(save_protos_path+'proto_lr'+str(lr_i)+'_wd'+str(wd_i)+'_birds.prototxt','w')
    temp_proto.writelines([line_p for line_p in lines_proto[0:40]])
    temp_proto.write('    lr_mult: '+str(lr_i)+'\n')
    temp_proto.write('    decay_mult: '+str(wd_i)+'\n')
    temp_proto.writelines([line_p for line_p in lines_proto[42:44]])
    temp_proto.write('    lr_mult: '+str(lr_i)+'\n')
    temp_proto.write('    decay_mult: '+str(wd_i)+'\n')
    temp_proto.writelines([line_p for line_p in lines_proto[46:69]])
    temp_proto.write('    lr_mult: '+str(lr_i)+'\n')
    temp_proto.write('    decay_mult: '+str(wd_i)+'\n')
    temp_proto.writelines([line_p for line_p in lines_proto[71:73]])
    temp_proto.write('    lr_mult: '+str(lr_i)+'\n')
    temp_proto.write('    decay_mult: '+str(wd_i)+'\n')
    temp_proto.writelines([line_p for line_p in lines_proto[75:len(lines_proto)]])

save_protos_path = '/home/jsotaloram/active_learning/exp_birds/protos/'

for (lr_i,wd_i) in hyper_params_combs:
    temp_proto = open(save_protos_path+'solver_proto_lr'+str(lr_i)+'_wd'+str(wd_i)+'_birds.prototxt','w')
    temp_proto.write('net: ' + '\"'+ save_protos_path+'proto_lr'+ str(lr_i)+'_wd' + str(wd_i)+'_birds.prototxt\"' + '\n')
    temp_proto.writelines([line_p for line_p in lines_solver[1:len(lines_solver)]])

#[line_p for line_p in lines_solver[0:10]]

generating prototxt for LR: 0.0001 and WD: 0.0001
generating prototxt for LR: 0.0001 and WD: 0.001
generating prototxt for LR: 0.0001 and WD: 0.005
generating prototxt for LR: 0.0001 and WD: 0.01
generating prototxt for LR: 0.0001 and WD: 0.1
generating prototxt for LR: 0.0001 and WD: 0.5
generating prototxt for LR: 0.0001 and WD: 1
generating prototxt for LR: 0.0001 and WD: 5
generating prototxt for LR: 0.0001 and WD: 2
generating prototxt for LR: 0.0001 and WD: 10
generating prototxt for LR: 0.0001 and WD: 100
generating prototxt for LR: 0.0003 and WD: 0.0001
generating prototxt for LR: 0.0003 and WD: 0.001
generating prototxt for LR: 0.0003 and WD: 0.005
generating prototxt for LR: 0.0003 and WD: 0.01
generating prototxt for LR: 0.0003 and WD: 0.1
generating prototxt for LR: 0.0003 and WD: 0.5
generating prototxt for LR: 0.0003 and WD: 1
generating prototxt for LR: 0.0003 and WD: 5
generating prototxt for LR: 0.0003 and WD: 2
generating prototxt for LR: 0.0003 and WD: 10
generating 

In [90]:
niter = 10000
for (lr_i,wd_i) in [(2,0.001)]:#hyper_params_combs[0:50]:
    temp_solver_path = save_protos_path+'solver_proto_lr'+str(lr_i)+'_wd'+str(wd_i)+'_birds.prototxt'
    print 'computing performance for solver: ' + temp_solver_path
    solver = cf.SGDSolver(temp_solver_path)
    train_loss = np.zeros(niter)
    test_loss = np.zeros(niter)
    scratch_train_loss = np.zeros(niter)
    num_hidden = num_hidden_neurons

    solver.net.set_input_arrays(data4D, data4DL)
    solver.test_nets[0].set_input_arrays(data4D_test,data4DL_test)

#It seems that labels should be of dimension (n, 1, 1, 1) for memory layer as well.


    for it in range(niter):
        solver.step(1)  # SGD by Caffe                                                                                                                     
        #scratch_solver.step(1)                                                                                                                            
        # store the train loss                                                                                                                             
        train_loss[it] = solver.net.blobs['loss'].data
        #scratch_train_loss[it] = scratch_solver.net.blobs['loss'].data                                                                                    
        test_loss[it] = solver.test_nets[0].blobs['loss'].data

        #if it % 10 == 0:
            #print 'iter %d, Training_Birds: hidden_%d_MLP_loss=%f' % (it,num_hidden_neurons, train_loss[it])
    #print 'done Training!'


    #reporting classification meassures of the small net:
    test_net = solver.test_nets[0] # more than one test net is supported

    test_net.set_input_arrays(data4D_test, data4DL_test)

    pred_labels_test_birds = []

    output = test_net.forward()

    for i in range(300):
        pred_labels_test_birds.append(test_net.blobs['prob'].data[i].flatten().argsort()[-1:-2:-1][0])



#CMSvmLfc7 = confusion_matrix(yTestLfc6_labels, pred_labels_test_birds, labels=birdsLabels)
#plt.figure(figsize=(8, 8))
#plot_confusion_matrix(CMSvmLfc7, labels=birdsLabels)

    #print(classification_report(yTestLfc6_labels, pred_labels_test_birds))
    cr_f1 = classification_report([yTestLfc6_labels[i] for i in full_idx_range_test], pred_labels_test_birds).split()
    print 'f1 so far: ' + str(cr_f1[-2])


#print solver.net.blobs['label'].data[29]
#output = solver.net.forward()
#print data4DL_test[29]
#output['prob'][29]
print train_loss
l_T = [yTestLfc6_labels[i] for i in full_idx_range_test]
print pred_labels_test_birds[0:30]
print l_T[0:30]

computing performance for solver: /home/jsotaloram/active_learning/exp_birds/protos/solver_proto_lr2_wd0.001_birds.prototxt
f1 so far: 0.21
[ 1.79380989  1.81274295  1.80461919 ...,  0.00706284  0.00886553
  0.00746444]
[2, 2, 1, 4, 0, 5, 2, 0, 4, 5, 4, 2, 2, 4, 4, 1, 2, 1, 0, 3, 2, 5, 1, 4, 3, 3, 1, 1, 4, 2]
[1, 0, 5, 1, 0, 5, 1, 2, 1, 0, 4, 3, 0, 4, 5, 4, 3, 1, 4, 5, 2, 2, 0, 1, 1, 3, 3, 5, 1, 2]


In [91]:
from scipy.special import expit as sigmoid
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import scipy.optimize


In [111]:
linSvmLfc7 = svm.LinearSVC()
linSvmLfc7.fit(XTrainLfc7, yTrainLfc7)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [98]:
yPredSvmLfc7 = linSvmLfc7.predict(XTestLf)
#CMSvmLfc7 = confusion_matrix(yTestLfc7, yPredSvmLfc7, labels=birdsLabels)
plt.figure(figsize=(8, 8))
print(classification_report(yTestLfc7, yPredSvmLfc7))
plt.figure(figsize=(8, 8))
plot_confusion_matrix(CMSvmLfc7, labels=birdsLabels)

             precision    recall  f1-score   support

      egret       0.17      0.20      0.18        50
   mandarin       0.16      0.20      0.18        50
        owl       0.22      0.26      0.24        50
     puffin       0.19      0.20      0.19        50
     toucan       0.12      0.06      0.08        50
  wood_duck       0.17      0.14      0.15        50

avg / total       0.17      0.18      0.17       300



NameError: name 'plot_confusion_matrix' is not defined

In [22]:
#reporting classification meassures of the small net:
test_net = solver.test_nets[0] # more than one test net is supported

test_net.set_input_arrays(data4D_test, data4DL_test)

pred_labels_test_birds = []

output = test_net.forward()
       
cur_lbls = []
for i in range(300):
    pred_labels_test_birds.append(test_net.blobs['prob'].data[i].flatten().argsort()[-1:-2:-1][0])

#print solver.net.blobs['label'].data[29]
#output = solver.net.forward()
#print data4DL_test[29]
#output['prob'][29]


In [28]:
print pred_labels_test_birds

[3, 5, 5, 2, 1, 0, 0, 4, 5, 1, 2, 0, 5, 0, 5, 1, 0, 0, 4, 5, 5, 2, 4, 5, 1, 0, 1, 3, 1, 4, 1, 5, 3, 0, 2, 4, 4, 0, 2, 4, 2, 1, 0, 4, 4, 3, 4, 0, 2, 5, 4, 0, 2, 4, 4, 4, 2, 5, 4, 2, 1, 1, 5, 4, 4, 1, 5, 5, 5, 2, 3, 0, 2, 5, 1, 0, 4, 1, 4, 2, 3, 4, 4, 5, 5, 1, 1, 1, 1, 1, 0, 4, 2, 2, 2, 1, 5, 5, 4, 2, 5, 5, 2, 0, 0, 1, 4, 4, 4, 2, 0, 1, 3, 3, 0, 5, 4, 2, 5, 0, 2, 1, 5, 5, 4, 0, 5, 0, 4, 3, 2, 4, 4, 0, 0, 3, 0, 4, 5, 0, 1, 1, 4, 4, 4, 1, 0, 1, 4, 0, 4, 5, 4, 1, 0, 1, 0, 5, 2, 3, 0, 3, 0, 4, 2, 4, 4, 0, 2, 4, 5, 2, 2, 5, 0, 5, 1, 2, 4, 2, 5, 1, 0, 1, 5, 1, 1, 1, 3, 1, 4, 3, 5, 1, 1, 1, 0, 1, 5, 5, 5, 5, 2, 0, 4, 3, 0, 1, 0, 5, 0, 3, 0, 4, 2, 4, 5, 3, 1, 1, 5, 3, 0, 3, 4, 0, 1, 1, 2, 4, 1, 4, 5, 2, 2, 3, 0, 0, 3, 4, 3, 4, 4, 4, 2, 1, 5, 3, 3, 5, 1, 4, 0, 4, 3, 5, 1, 1, 0, 1, 2, 0, 2, 5, 2, 3, 0, 4, 1, 1, 1, 1, 4, 2, 4, 4, 3, 1, 3, 1, 4, 2, 3, 4, 4, 5, 1, 4, 1, 2, 5, 4, 5, 5, 3, 2, 5, 1, 5, 4]


In [41]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=birdsLabels):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=90)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [43]:
# generate dictionaries of the parameters
params = solver.net.params.keys()
source_params = {pr: (solver.net.params[pr][0].data, solver.net.params[pr][1].data) for pr in params}
target_params = {pr: (solver.net.params[pr][0].data, solver.net.params[pr][1].data) for pr in params}
#print source_params
#print target_params


In [77]:

birdsFolder = "/data1/birds"

birdsLabels = os.listdir(birdsFolder)

CMSvmLfc7 = confusion_matrix(yTestLfc6_labels, pred_labels_test_birds, labels=birdsLabels)
plt.figure(figsize=(8, 8))
#plot_confusion_matrix(CMSvmLfc7, labels=birdsLabels)
cats_test_idx = [all_samples_cats[i] for i in testIdx]
print(classification_report(yTestLfc7, [dict_cats_nameOnly[i] for i in cats_test_idx]))
#[inv_bird_map_nameOnly[i] for i in all_samples_cats[testIdx]]
#print(classification_report(yTestLfc6_labels, pred_labels_test_birds))
#cr_f1 = classification_report(yTestLfc6_labels, pred_labels_test_birds).split()
#cr_f1[-2]
#all_samples_cats[test_idx]
#yTestLfc7
pred_labels_test_birds

             precision    recall  f1-score   support

      egret       1.00      1.00      1.00        50
   mandarin       1.00      1.00      1.00        50
        owl       1.00      1.00      1.00        50
     puffin       1.00      1.00      1.00        50
     toucan       1.00      1.00      1.00        50
  wood_duck       1.00      1.00      1.00        50

avg / total       1.00      1.00      1.00       300



[3,
 5,
 5,
 2,
 1,
 0,
 0,
 4,
 5,
 1,
 2,
 0,
 5,
 0,
 5,
 1,
 0,
 0,
 4,
 5,
 5,
 2,
 4,
 5,
 1,
 0,
 1,
 3,
 1,
 4,
 1,
 5,
 3,
 0,
 2,
 4,
 4,
 0,
 2,
 4,
 2,
 1,
 0,
 4,
 4,
 3,
 4,
 0,
 2,
 5,
 4,
 0,
 2,
 4,
 4,
 4,
 2,
 5,
 4,
 2,
 1,
 1,
 5,
 4,
 4,
 1,
 5,
 5,
 5,
 2,
 3,
 0,
 2,
 5,
 1,
 0,
 4,
 1,
 4,
 2,
 3,
 4,
 4,
 5,
 5,
 1,
 1,
 1,
 1,
 1,
 0,
 4,
 2,
 2,
 2,
 1,
 5,
 5,
 4,
 2,
 5,
 5,
 2,
 0,
 0,
 1,
 4,
 4,
 4,
 2,
 0,
 1,
 3,
 3,
 0,
 5,
 4,
 2,
 5,
 0,
 2,
 1,
 5,
 5,
 4,
 0,
 5,
 0,
 4,
 3,
 2,
 4,
 4,
 0,
 0,
 3,
 0,
 4,
 5,
 0,
 1,
 1,
 4,
 4,
 4,
 1,
 0,
 1,
 4,
 0,
 4,
 5,
 4,
 1,
 0,
 1,
 0,
 5,
 2,
 3,
 0,
 3,
 0,
 4,
 2,
 4,
 4,
 0,
 2,
 4,
 5,
 2,
 2,
 5,
 0,
 5,
 1,
 2,
 4,
 2,
 5,
 1,
 0,
 1,
 5,
 1,
 1,
 1,
 3,
 1,
 4,
 3,
 5,
 1,
 1,
 1,
 0,
 1,
 5,
 5,
 5,
 5,
 2,
 0,
 4,
 3,
 0,
 1,
 0,
 5,
 0,
 3,
 0,
 4,
 2,
 4,
 5,
 3,
 1,
 1,
 5,
 3,
 0,
 3,
 4,
 0,
 1,
 1,
 2,
 4,
 1,
 4,
 5,
 2,
 2,
 3,
 0,
 0,
 3,
 4,
 3,
 4,
 4,
 4,
 2,
 1,
 5,
 3,
 3,
 5,


In [127]:
#Doing a K-means over the feats to initialize weights:
from sklearn.cluster import KMeans



In [61]:
#Extracting the gradients of the batch forward pass


all_shapes = [(k, v.data.shape) for k, v in test_net.blobs.items()]
print all_shapes

[('data', (300, 1, 1, 4096)), ('label', (300, 1, 1, 1)), ('label_data_1_split_0', (300, 1, 1, 1)), ('label_data_1_split_1', (300, 1, 1, 1)), ('fc7', (300, 32)), ('fc8_birds', (300, 6)), ('fc8_birds_fc8_birds_0_split_0', (300, 6)), ('fc8_birds_fc8_birds_0_split_1', (300, 6)), ('fc8_birds_fc8_birds_0_split_2', (300, 6)), ('prob', (300, 6)), ('loss', ()), ('accuracy', ())]


In [ ]:
#Using EGC

#Compare against RL


In [4]:
lines_proto = []
f = open(deployPrototxt,'r')
for line in f:
    lines_proto.append(line)
    
learning_rates_pool = [0.0001,0.001,0.01,0.1,1,10,100]
decay_mult = [0.0001,0.001,0.01,0.1,1,10,100]

[line_p for line_p in lines_proto[0:10]]

['name: "birds_small_caffe"\n',
 'layer {\n',
 '  name: "data"\n',
 '  type: "MemoryData"\n',
 '  top: "data"\n',
 '  top: "label"\n',
 '  memory_data_param {\n',
 '    batch_size: 30\n',
 '    channels: 1\n',
 '    height: 1\n']

In [1]:
save_protos_path = '/home/jsotaloram/active_learning/exp_birds/protos/'
hyper_params_combs = [] 
for lr_item in learning_rates_pool:
    for dec_item in decay_mult:
        #print 'combination of lr: ' + str(lr_item) +' and decay: ' +str(dec_item) + '\n'
        hyper_params_combs.append((lr_item,dec_item))
#for i in range(len(lines_proto)):
#    print 'Linea : ' + str(i)+lines_proto[i] + '\n\n'
    
hyper_params_combs
lines_proto[40].split()

NameError: name 'learning_rates_pool' is not defined